# 04-工具调用

Function Call 和官方工具使用

In [ ]:
from openai import OpenAI
import json
import os

client = OpenAI(
    api_key=os.getenv("MOONSHOT_API_KEY", "your-api-key"),
    base_url="https://api.moonshot.cn/v1",
)

## 基础工具调用

In [ ]:
# 定义工具
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取指定城市的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "城市名称"}
                },
                "required": ["city"]
            }
        }
    }
]

# 调用
response = client.chat.completions.create(
    model="kimi-k2-turbo-preview",
    messages=[{"role": "user", "content": "北京天气如何？"}],
    tools=tools,
    tool_choice="auto"
)

message = response.choices[0].message

if message.tool_calls:
    print(f"需要调用工具: {message.tool_calls[0].function.name}")
    print(f"参数: {message.tool_calls[0].function.arguments}")
else:
    print(f"直接回答: {message.content}")

## 完整工具调用流程

In [ ]:
# 步骤 1: 第一次调用
messages = [{"role": "user", "content": "北京天气如何？"}]

response = client.chat.completions.create(
    model="kimi-k2-turbo-preview",
    messages=messages,
    tools=tools
)

assistant_message = response.choices[0].message
messages.append(assistant_message)

# 步骤 2: 执行工具并返回结果
if assistant_message.tool_calls:
    tool_call = assistant_message.tool_calls[0]
    city = json.loads(tool_call.function.arguments)["city"]
    
    # 模拟工具执行
    weather_result = f"{city}今天晴，25°C，空气质量优"
    print(f"工具执行: 获取{city}天气 = {weather_result}")
    
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": weather_result
    })
    
    # 步骤 3: 获取最终回答
    final_response = client.chat.completions.create(
        model="kimi-k2-turbo-preview",
        messages=messages,
        tools=tools
    )
    
    print(f"\n最终回答: {final_response.choices[0].message.content}")

## 官方内置工具 - 联网搜索

注意：思考模式与 `$web_search` 不兼容，需关闭思考

In [ ]:
# 使用官方 $web_search 工具
tools = [
    {
        "type": "builtin_function",
        "function": {"name": "$web_search"}
    }
]

response = client.chat.completions.create(
    model="kimi-k2.5",
    messages=[{"role": "user", "content": "今天有什么新闻？"}],
    tools=tools,
    extra_body={"thinking": {"type": "disabled"}}  # 关闭思考模式
)

print(response.choices[0].message.content)

## 思考模型 + 工具调用（多步）

In [ ]:
# 多步工具调用
tools = [
    {"type": "function", "function": {"name": "date", "description": "获取日期"}},
    {"type": "function", "function": {"name": "weather", "description": "获取天气"}}
]

response = client.chat.completions.create(
    model="kimi-k2-thinking",
    messages=[{"role": "user", "content": "今天天气如何？"}],
    tools=tools,
    max_tokens=16000,
    temperature=1.0,
    stream=True
)

for chunk in response:
    delta = chunk.choices[0].delta
    
    reasoning = getattr(delta, "reasoning_content", None)
    if reasoning:
        print(f"[思考] {reasoning}", end="")
    
    if delta.content:
        print(delta.content, end="")

print()